In [1]:
import pandas as pd
from IPython.display import display
import datetime
from copy import deepcopy

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
census_data = pd.read_csv("acs2017_census_tract_data.csv")
census_cols = list(census_data.columns)
census_data = census_data.iloc[:, [census_cols.index("State"), census_cols.index("County"), census_cols.index("TotalPop"), census_cols.index("Poverty"), census_cols.index("IncomePerCap")]]
covid_data = pd.read_csv("COVID_county_data.csv")

In [4]:
census_cols

['TractId',
 'State',
 'County',
 'TotalPop',
 'Men',
 'Women',
 'Hispanic',
 'White',
 'Black',
 'Native',
 'Asian',
 'Pacific',
 'VotingAgeCitizen',
 'Income',
 'IncomeErr',
 'IncomePerCap',
 'IncomePerCapErr',
 'Poverty',
 'ChildPoverty',
 'Professional',
 'Service',
 'Office',
 'Construction',
 'Production',
 'Drive',
 'Carpool',
 'Transit',
 'Walk',
 'OtherTransp',
 'WorkAtHome',
 'MeanCommute',
 'Employed',
 'PrivateWork',
 'PublicWork',
 'SelfEmployed',
 'FamilyWork',
 'Unemployment']

In [5]:
covid_data = covid_data.rename(columns={"state": "State", "county": "County", "cases": "TotCases", "deaths": "TotDeaths"})
covid_data["date"] = pd.to_datetime(covid_data["date"])
covid_data = covid_data.drop(["fips"], axis=1)

In [6]:
covid_data[["date"]].head(10)

,date
0,2020-01-21
1,2020-01-22
2,2020-01-23
3,2020-01-24
4,2020-01-24
5,2020-01-25
6,2020-01-25
7,2020-01-25
8,2020-01-26
9,2020-01-26


In [7]:
census_data["TotIncomeCounty"] = census_data["TotalPop"] * census_data["IncomePerCap"]
census_data[["TotIncomeCounty"]].head()

,TotIncomeCounty
0,60918210.0
1,41259312.0
2,71883860.0
3,119766156.0
4,367758325.0


In [8]:
census_data["PovertyPop"] = census_data["TotalPop"] * census_data["Poverty"] / 100
census_data[["PovertyPop"]].head()

,PovertyPop
0,197.415
1,486.528
2,497.595
3,98.141
4,1215.730


In [9]:
city_locs = [("Oregon", "Washington"), ("Virginia", "Loudon"), ("Kentucky", "Harlan"), ("Oregon", "Malheur")]

In [10]:
a = census_data.groupby(["State", "County"])
for cl in city_locs:
    try:
        display(a.get_group(cl).groupby(["State", "County"]).sum())
    except KeyError:
        print(f"Couldnt find data for {cl}")

,,TotalPop,Poverty,IncomePerCap,TotIncomeCounty,PovertyPop
State,County,,,,,
Oregon,Washington,572071,1086.4,3636965.0,2.023361e+10,59044.602


Couldnt find data for ('Virginia', 'Loudon')


,,TotalPop,Poverty,IncomePerCap,TotIncomeCounty,PovertyPop
State,County,,,,,
Kentucky,Harlan,27548,366.5,176114.0,425808638.0,9826.229


,,TotalPop,Poverty,IncomePerCap,TotIncomeCounty,PovertyPop
State,County,,,,,
Oregon,Malheur,30421,170.9,125765.0,534421049.0,7391.763


In [11]:
### Dec2020 cases

choose_year = 2020
choose_month = 12

row_indxs = covid_data["date"].apply(
    lambda gvn: gvn.year == choose_year and gvn.month == choose_month
)

dec_2020 = covid_data[row_indxs].groupby(["State", "County"]).sum()
dec_2020 = dec_2020.rename(columns={"TotCases": "Dec2020Cases", "TotDeaths": "Dec2020Deats"})

In [12]:
dec_2020

Dec2020Cases  Dec2020Deats
State   County                                
Alabama Autauga           108652        1355.0
        Baldwin           348455        4502.0
        Barbour            40753         931.0
        Bibb               47009        1244.0
        Blount            121270        1590.0
...                          ...           ...
Wyoming Sweetwater         78122         412.0
        Teton              59845          67.0
        Uinta              41859         201.0
        Washakie           20107         350.0
        Weston             13656          62.0

[3247 rows x 2 columns]

In [13]:
covid_combined = pd.merge(covid_data.groupby(["State", "County"]).sum(), dec_2020, how ='inner', on=["State", "County"])
covid_combined

TotCases  TotDeaths  Dec2020Cases  Dec2020Deats
State   County                                                     
Alabama Autauga       645935     9042.0        108652        1355.0
        Baldwin      2003567    23041.0        348455        4502.0
        Barbour       268771     4077.0         40753         931.0
        Bibb          261043     5272.0         47009        1244.0
        Blount        630106     8669.0        121270        1590.0
...                      ...        ...           ...           ...
Wyoming Sweetwater    323730     2239.0         78122         412.0
        Teton         305376      617.0         59845          67.0
        Uinta         200783     1037.0         41859         201.0
        Washakie       84354     2622.0         20107         350.0
        Weston         56976      286.0         13656          62.0

[3247 rows x 4 columns]

In [14]:
b = covid_combined.groupby(["State", "County"])
for cl in city_locs:
    try:
        display(b.get_group(cl).groupby(["State", "County"]).sum())
    except KeyError:
        print(f"Couldnt find data for {cl}")

,,TotCases,TotDeaths,Dec2020Cases,Dec2020Deats
State,County,,,,
Oregon,Washington,2157339,22455.0,424620,3860.0


Couldnt find data for ('Virginia', 'Loudon')


,,TotCases,TotDeaths,Dec2020Cases,Dec2020Deats
State,County,,,,
Kentucky,Harlan,205984,3994.0,38959,506.0


,,TotCases,TotDeaths,Dec2020Cases,Dec2020Deats
State,County,,,,
Oregon,Malheur,453634,7770.0,82916,1465.0


In [15]:
joined_data = pd.merge(census_data.groupby(["State", "County"]).sum(), covid_combined, how ='inner', on=["State", "County"])
joined_data

TotalPop  Poverty  IncomePerCap  TotIncomeCounty  \
State   County                                                         
Alabama Autauga        55036    174.7      319058.0     1.531317e+09   
        Baldwin       203360    399.1      903052.0     5.971537e+09   
        Barbour        26201    249.8      161025.0     4.601180e+08   
        Bibb           22580     55.7       87196.0     4.721744e+08   
        Blount         57667    147.8      194386.0     1.269869e+09   
...                      ...      ...           ...              ...   
Wyoming Sweetwater     44527    140.3      394777.0     1.411504e+09   
        Teton          22923     30.2      190825.0     1.127826e+09   
        Uinta          20758     44.2       81258.0     5.628499e+08   
        Washakie        8253     39.2       80485.0     2.256782e+08   
        Weston          7117     28.4       62117.0     2.203060e+08   

                    PovertyPop  TotCases  TotDeaths  Dec2020Cases  \
State   County                                                      
Alabama Autauga       7570.912    645935     9042.0        108652   
        Baldwin      24138.132   2003567    23041.0        348455   
        Barbour       7039.682    268771     4077.0         40753   
        Bibb          3368.040    261043     5272.0         47009   
        Blount        8998.906    630106     8669.0        121270   
...                        ...       ...        ...           ...   
Wyoming Sweetwater    5378.688    323730     2239.0         78122   
        Teton         1568.565    305376      617.0         59845   
        Uinta         3076.634    200783     1037.0         41859   
        Washakie      1061.903     84354     2622.0         20107   
        Weston        1018.417     56976      286.0         13656   

                    Dec2020Deats  
State   County                    
Alabama Autauga           1355.0  
        Baldwin           4502.0  
        Barbour            931.0  
        Bibb              1244.0  
        Blount            1590.0  
...                          ...  
Wyoming Sweetwater         412.0  
        Teton               67.0  
        Uinta              201.0  
        Washakie           350.0  
        Weston              62.0  

[3068 rows x 9 columns]

In [16]:
joined_data["PerCapitaIncome"] = joined_data["TotIncomeCounty"] / joined_data["TotalPop"]
joined_data["Poverty"] = joined_data["PovertyPop"] * 100 / joined_data["TotalPop"]

joined_data = joined_data.drop(["TotIncomeCounty", "PovertyPop"], axis=1)
joined_data

TotalPop    Poverty  IncomePerCap  TotCases  TotDeaths  \
State   County                                                               
Alabama Autauga        55036  13.756290      319058.0    645935     9042.0   
        Baldwin       203360  11.869656      903052.0   2003567    23041.0   
        Barbour        26201  26.867990      161025.0    268771     4077.0   
        Bibb           22580  14.916032       87196.0    261043     5272.0   
        Blount         57667  15.604949      194386.0    630106     8669.0   
...                      ...        ...           ...       ...        ...   
Wyoming Sweetwater     44527  12.079610      394777.0    323730     2239.0   
        Teton          22923   6.842756      190825.0    305376      617.0   
        Uinta          20758  14.821438       81258.0    200783     1037.0   
        Washakie        8253  12.866873       80485.0     84354     2622.0   
        Weston          7117  14.309639       62117.0     56976      286.0   

                    Dec2020Cases  Dec2020Deats  PerCapitaIncome  
State   County                                                   
Alabama Autauga           108652        1355.0     27823.920161  
        Baldwin           348455        4502.0     29364.365583  
        Barbour            40753         931.0     17561.086981  
        Bibb               47009        1244.0     20911.176616  
        Blount            121270        1590.0     22020.719562  
...                          ...           ...              ...  
Wyoming Sweetwater         78122         412.0     31699.951737  
        Teton              59845          67.0     49200.625311  
        Uinta              41859         201.0     27114.842808  
        Washakie           20107         350.0     27344.988853  
        Weston             13656          62.0     30954.891246  

[3068 rows x 8 columns]

In [17]:
c = joined_data.groupby(["State"])
for cl in [("Oregon")]:
    try:
        display(c.get_group(cl).groupby(["State", "County"]).sum())
    except KeyError:
        print(f"Couldnt find data for {cl}")

TotalPop    Poverty  IncomePerCap  TotCases  TotDeaths  \
State  County                                                               
Oregon Baker          15980  15.083855      154241.0     55586      663.0   
       Benton         88249  22.421152      538668.0    180225     2304.0   
       Clackamas     399962   8.976120     3000217.0   1284402    20040.0   
       Clatsop        38021  12.190090      311931.0     77666      287.0   
       Columbia       50207  12.315329      281097.0    105324     1363.0   
       Coos           62921  17.896488      344348.0    100097      969.0   
       Crook          21717  15.320864       95834.0     55863     1134.0   
       Curry          22377  15.408656      134496.0     30045      393.0   
       Deschutes     175321  12.100898      764025.0    509974     4141.0   
       Douglas       107576  17.025995      554588.0    174952     3983.0   
       Gilliam         1910   9.900000       24178.0      4691       76.0   
       Grant           7209  13.635802       47710.0     18551       94.0   
       Harney          7195  17.528770       50349.0     17024      291.0   
       Hood River     22938  12.123145      116712.0    107383     1444.0   
       Jackson       212070  16.858350     1120480.0    713288     7221.0   
       Jefferson      22707  20.694856      136138.0    200346     2630.0   
       Josephine      84514  18.646376      386865.0    153675     2638.0   
       Klamath        66018  18.688624      474248.0    224256     2857.0   
       Lake            7807  20.139311       42243.0     25357      348.0   
       Lane          363471  19.230471     2368975.0    850956    10372.0   
       Lincoln        47307  18.376280      455726.0    153979     3117.0   
       Linn          121074  16.063929      513507.0    324636     5949.0   
       Malheur        30421  24.298225      125765.0    453634     7770.0   
       Marion        330453  16.128516     1502424.0   1974030    34089.0   
       Morrow         11153  14.699050       46343.0    139209     1447.0   
       Multnomah     788459  16.474668     6245725.0   3374737    58787.0   
       Polk           79666  15.639958      295607.0    268036     5480.0   
       Sherman         1635  13.700000       34226.0      5807        0.0   
       Tillamook      25840  15.512717      206446.0     34370       92.0   
       Umatilla       76736  17.825222      348007.0    933975    10661.0   
       Union          25810  17.618597      212071.0    161223     1533.0   
       Wallowa         6864  13.748776       80829.0     13017      449.0   
       Wasco          25687  13.670818      200718.0    121202     3039.0   
       Washington    572071  10.321202     3636965.0   2157339    22455.0   
       Wheeler         1415  20.600000       21268.0      1454       53.0   
       Yamhill       102366  13.802658      485841.0    356425     6010.0   

                   Dec2020Cases  Dec2020Deats  PerCapitaIncome  
State  County                                                   
Oregon Baker              11688         133.0     25820.273154  
       Benton             34260         278.0     30872.824361  
       Clackamas         261810        3125.0     37550.849108  
       Clatsop            14439          47.0     28114.625523  
       Columbia           21459         266.0     28459.688051  
       Coos               18806         151.0     26007.212997  
       Crook              11048         196.0     24238.814477  
       Curry               6741          72.0     26925.536399  
       Deschutes         102490         563.0     31574.934092  
       Douglas            37590         964.0     25001.732924  
       Gilliam              898          25.0     24178.000000  
       Grant               4895          31.0     25154.161742  
       Harney              3717          34.0     24397.712578  
       Hood River         19348         216.0     29594.972796  
       Jackson           154535        1655.0     27

In [18]:
normalized_data = deepcopy(joined_data)

In [19]:
normalized_data["NormalizedCovidCases"] = normalized_data["TotCases"] * 100000 / normalized_data["TotalPop"]
normalized_data["NormalizedCovidDeaths"] = normalized_data["TotDeaths"] * 100000 / normalized_data["TotalPop"]
normalized_data["NormalizedDec2020Cases"] = normalized_data["Dec2020Cases"] * 100000 / normalized_data["TotalPop"]
normalized_data["NormalizedDec2020Deaths"] = normalized_data["Dec2020Deats"] * 100000 / normalized_data["TotalPop"]

In [20]:
### Across All Counties in state of Oregon

need_data = normalized_data.groupby(["State"])
need_data = need_data.get_group("Oregon")

## COVID total cases vs Poverty Population Percentage

R = need_data["NormalizedCovidCases"].corr(need_data["Poverty"])
print(f"COVID total cases vs Poverty Population Percentage: {R}")

## COVID total deaths vs Poverty Population Percentage

R = need_data["NormalizedCovidDeaths"].corr(need_data["Poverty"])
print(f"COVID total cases vs Poverty Population Percentage: {R}")

## COVID total cases vs Per Capita Income

R = need_data["NormalizedCovidCases"].corr(need_data["PerCapitaIncome"])
print(f"COVID total cases vs Per Capita Income: {R}")

## COVID total deaths vs Poverty Population Percentage

R = need_data["NormalizedCovidDeaths"].corr(need_data["PerCapitaIncome"])
print(f"COVID total deaths vs Per Capita Income: {R}")

## COVID DEC 2020 cases vs Poverty Population Percentage

R = need_data["NormalizedDec2020Cases"].corr(need_data["Poverty"])
print(f"COVID DEC 2020 cases vs Poverty Population Percentage: {R}")

## COVID DEC 2020 deaths vs Poverty Population Percentage

R = need_data["NormalizedDec2020Deaths"].corr(need_data["Poverty"])
print(f"COVID DEC 2020 cases vs Poverty Population Percentage: {R}")

## COVID DEC 2020 cases vs Per Capita Income

R = need_data["NormalizedDec2020Cases"].corr(need_data["PerCapitaIncome"])
print(f"COVID DEC 2020 cases vs Per Capita Income: {R}")

## COVID DEC 2020 deaths vs Poverty Population Percentage

R = need_data["NormalizedDec2020Deaths"].corr(need_data["PerCapitaIncome"])
print(f"COVID DEC 2020 deaths vs Per Capita Income: {R}")

COVID total cases vs Poverty Population Percentage: 0.28707860802137736
COVID total cases vs Poverty Population Percentage: 0.3605391158241334
COVID total cases vs Per Capita Income: -0.37568502761471967
COVID total deaths vs Per Capita Income: -0.4618665950518555
COVID DEC 2020 cases vs Poverty Population Percentage: 0.29815203013315383
COVID DEC 2020 cases vs Poverty Population Percentage: 0.3027269512831472
COVID DEC 2020 cases vs Per Capita Income: -0.3853971943730501
COVID DEC 2020 deaths vs Per Capita Income: -0.45595519506866566


In [21]:
### Across All Counties in USA

## COVID total cases vs Poverty Population Percentage

R = normalized_data["NormalizedCovidCases"].corr(normalized_data["Poverty"])
print(f"COVID total cases vs Poverty Population Percentage: {R}")

## COVID total deaths vs Poverty Population Percentage

R = normalized_data["NormalizedCovidDeaths"].corr(normalized_data["Poverty"])
print(f"COVID total cases vs Poverty Population Percentage: {R}")

## COVID total cases vs Per Capita Income

R = normalized_data["NormalizedCovidCases"].corr(normalized_data["PerCapitaIncome"])
print(f"COVID total cases vs Per Capita Income: {R}")

## COVID total deaths vs Poverty Population Percentage

R = normalized_data["NormalizedCovidDeaths"].corr(normalized_data["PerCapitaIncome"])
print(f"COVID total deaths vs Per Capita Income: {R}")

## COVID DEC 2020 cases vs Poverty Population Percentage

R = normalized_data["NormalizedDec2020Cases"].corr(normalized_data["Poverty"])
print(f"COVID DEC 2020 cases vs Poverty Population Percentage: {R}")

## COVID DEC 2020 deaths vs Poverty Population Percentage

R = normalized_data["NormalizedDec2020Deaths"].corr(normalized_data["Poverty"])
print(f"COVID DEC 2020 cases vs Poverty Population Percentage: {R}")

## COVID DEC 2020 cases vs Per Capita Income

R = normalized_data["NormalizedDec2020Cases"].corr(normalized_data["PerCapitaIncome"])
print(f"COVID DEC 2020 cases vs Per Capita Income: {R}")

## COVID DEC 2020 deaths vs Poverty Population Percentage

R = normalized_data["NormalizedDec2020Deaths"].corr(normalized_data["PerCapitaIncome"])
print(f"COVID DEC 2020 deaths vs Per Capita Income: {R}")

COVID total cases vs Poverty Population Percentage: 0.19743877808931917
COVID total cases vs Poverty Population Percentage: 0.2680378406210281
COVID total cases vs Per Capita Income: -0.2136606336062747
COVID total deaths vs Per Capita Income: -0.1734583500312029
COVID DEC 2020 cases vs Poverty Population Percentage: 0.07036431525535303
COVID DEC 2020 cases vs Poverty Population Percentage: 0.20267649352851475
COVID DEC 2020 cases vs Per Capita Income: -0.1593355849426528
COVID DEC 2020 deaths vs Per Capita Income: -0.18746387035688974
